Heston Model

In [28]:
import numpy as np
i = complex(0, 1)

# To be used in the Heston pricer
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s

    # Calculate d
    d1 = (prod - kappa)**2
    d2 = (sigma**2) * (i * s + s**2)
    d = np.sqrt(d1 + d2)

    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2

    # Calculate first exponential
    exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, -2 * theta * kappa/(sigma**2))

    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))
    return (mainExp1 * mainExp2)

# Heston Pricer
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations
    element1 = 0.5 * (St - K * np.exp(-r * T))

    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        numerator1 = fHeston(s2, St, K, r, T, 
                             sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, 
                               sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1
        P += ds * (numerator1 - numerator2) / denominator
        
    element2 = P / np.pi
    return np.real((element1 + element2))

Black Scholes Model and Jump Diffusion Model

In [126]:
from scipy.stats import norm

N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)    
    
    
def merton_jump_call(S, K, T, r, sigma, m , v, lam):
    p = 0

    for k in range(5):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        
        k_fact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * BS_CALL(S, K, T, r_k, sigma_k)
    
    return p 

def merton_jump_put(S, K, T, r, sigma, m , v, lam):
    p = 0 # price of option
    for k in range(40):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k) # 
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact)) \
                    * BS_PUT(S, K, T, r_k, sigma_k)
    return p 

In [127]:
S = 100
K = 100
T = 1
r = 0.1
sigma = 0.1
m = 1 # meean of jump size
v = 0.1 # standard deviation of jump
lam = 1 # intensity of jump i.e. number of jumps per annum
sigma = 0.1 # annaul standard deviation , for weiner process


#print(BS_CALL(S, K, T, r, sigma))
#print(BS_PUT(S, K, T, r, sigma))

merton_jump_call(S, K, T, r, sigma, m , v, lam)

11.299459752587682

In [110]:
k = 3
np.exp(-m*lam*T) * (m*lam*T)**k / (np.math.factorial(k))

0.36787944117144233

In [32]:
# Heston Parameters
sigma = 0.8 # volatility 
kappa = 0.2 # kappa = ?
theta = 0.1 # theta = ?
volvol = 0.5 # volatility of volatility
rho = 0.4 # rho = ?

St = 100 # current stock price
K = 99.9
T = 0.5 # time to maturity
r = 0.2 # risk free rate
m = 0 # meean of jump size
v = 0.3 # standard deviation of jump
lam = 1 # intensity of jump i.e. number of jumps per annum
sigma = 0.8 # annaul standard deviation , for weiner process

cf_price =  merton_jump_call(St, K, T, r, sigma, np.exp(m+v**2*0.5) , v, lam)
c = priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho)

print('Heston Price =', c)
print('Merton Price =', cf_price)
print('Black Scholes Price =', BS_CALL(St,K,T,r, sigma))

Heston Price = 24.250253497476706
Merton Price = 27.671047232833228
Black Scholes Price = 26.33169186043942


## Rewritten to not use any libraries or dependencies (except numpy)

sources:

In [33]:
# https://www.johndcook.com/blog/2009/01/19/stand-alone-error-function-erf/
# https://stats.stackexchange.com/questions/187828/how-are-the-error-function-and-standard-normal-distribution-function-related
# https://www.johndcook.com/erf_and_normal_cdf.pdf

In [34]:
import numpy as np

norm.pdf as vanilla python function:

In [35]:
def pdf(x):
    y = np.exp((-1 * x**2)/2) / np.sqrt(2*np.pi)
    return y

In [36]:
print(pdf(-1))
print(norm.pdf(-1))

0.24197072451914337
0.24197072451914337


Error Function:

In [37]:
import math

def erf(x):
    # constants
    a1 =  0.254829592
    a2 = -0.284496736
    a3 =  1.421413741
    a4 = -1.453152027
    a5 =  1.061405429
    p  =  0.3275911

    # Save the sign of x
    sign = 1
    if x < 0:
        sign = -1
    x = abs(x)

    # A & S 7.1.26
    t = 1.0/(1.0 + p*x)
    y = 1.0 - (((((a5*t + a4)*t) + a3)*t + a2)*t + a1)*t*math.exp(-x*x)

    return sign*y

In [38]:
erf(0.5)

0.5205000163047472

In [39]:
def erf(x):
    # constants
    a1 =  254829592000000000
    a2 = -284496736000000000
    a3 =  1421413741000000000
    a4 = -1453152027000000000
    a5 =  1061405429000000000
    p  =  327591100000000000

    # Save the sign of x
    sign = 1
    if x < 0:
        sign = -1
    x = abs(x)

    # A & S 7.1.26
    t = 1e36/ (1e18 + (p*x) / 1e18)
    y = 1e18 - ((((((a5*t/1e18 + a4)*t/1e18) + a3)*t/1e18 + a2)*t/1e18 + a1)*t/1e18) * math.exp((-x*x)/1e36)

    return sign*y

In [40]:
erf(5e17)

5.20500016304747e+17

Converting Error Function to CDF function: 

source: https://www.johndcook.com/erf_and_normal_cdf.pdf

In [41]:
import math

def erf(x):
    # constants
    a1 =  0.254829592
    a2 = -0.284496736
    a3 =  1.421413741
    a4 = -1.453152027
    a5 =  1.061405429
    p  =  0.3275911

    # Save the sign of x
    sign = 1
    if x < 0:
        sign = -1
    x = abs(x)

    # A & S 7.1.26
    t = 1.0/(1.0 + p*x)
    y = 1.0 - (((((a5*t + a4)*t) + a3)*t + a2)*t + a1)*t*math.exp(-x*x)

    return sign*y



def cdf(x):
    y = (erf(x / np.sqrt(2)) + 1) / 2
    return y

In [42]:
cdf(0.5)

0.6914624627239938

In [43]:
N = norm.cdf
N(1)

0.8413447460685429

In [44]:
def erf(x):
    # constants
    a1 =  254829592000000000
    a2 = -284496736000000000
    a3 =  1421413741000000000
    a4 = -1453152027000000000
    a5 =  1061405429000000000
    p  =  327591100000000000

    # Save the sign of x
    sign = 1
    if x < 0:
        sign = -1
    x = abs(x)

    # A & S 7.1.26
    t = 1e36/ (1e18 + (p*x) / 1e18)
    y = 1e18 - ((((((a5*t/1e18 + a4)*t/1e18) + a3)*t/1e18 + a2)*t/1e18 + a1)*t/1e18) * math.exp((-x*x)/1e36)

    return sign*y

def cdf(x):
    y = (erf(x * 1e18 / 1414213562373095100) + 1e18) / 2
    return y

In [45]:
cdf(0.005*1e18)

5.019947155784597e+17

Rewritten BS functions:

In [46]:
def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * cdf(d1) - K * np.exp(-r*T)* cdf(d2)


def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*cdf(-d2) - S*cdf(-d1)   


def merton_jump_call(S, K, T, r, sigma, m , v, lam):
    p = 0
    for k in range(40):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k)
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact))  * BS_CALL(S, K, T, r_k, sigma_k)
    
    return p 

def merton_jump_put(S, K, T, r, sigma, m , v, lam):
    p = 0 # price of option
    for k in range(40):
        r_k = r - lam*(m-1) + (k*np.log(m) ) / T
        sigma_k = np.sqrt( sigma**2 + (k* v** 2) / T)
        k_fact = np.math.factorial(k) # 
        p += (np.exp(-m*lam*T) * (m*lam*T)**k / (k_fact)) \
                    * BS_PUT(S, K, T, r_k, sigma_k)
    return p

In [47]:
S = 100
K = 100
T = 1
r = 0.1
sigma = 0.1

BS_CALL(S, K, T, r, sigma)

4.758129102960157e+18

In [ ]:
def erf(x):
    # constants
    a1 =  254829592000000000
    a2 = -284496736000000000
    a3 =  1421413741000000000
    a4 = -1453152027000000000
    a5 =  1061405429000000000
    p  =  327591100000000000

    # Save the sign of x
    sign = 1
    if x < 0:
        sign = -1
    x = abs(x)

    # A & S 7.1.26
    t = 1e36/ (1e18 + (p*x) / 1e18)
    y = 1e18 - ((((((a5*t/1e18 + a4)*t/1e18) + a3)*t/1e18 + a2)*t/1e18 + a1)*t/1e18) * math.exp((-x*x)/1e36)

    return sign*y

def cdf(x):
    y = (erf(x * 1e18 / 1414213562373095100) + 1e18) / 2
    return y


def BS_CALL(S, K, T, r, sigma):
    d1 = d_1(S,K,T,r,sigma)
    d2 = d_2(d1,sigma,T)
    C = c(d1,d2,S,K,T,r,sigma)
    return C

def d_1(S,K,T,r,sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2 / 1e18)*T / 1e18) / (sigma*np.sqrt(T) / 1e27)
    return d1

def d_2(d1,sigma,T):
    d2 = d1 - sigma * np.sqrt(T/1e18) 
    return d2

def c(d1,d2,S,K,T,r,sigma):
    C = S * cdf(d1) / 1e18 - K * np.exp(-r*T/1e18 / 1e18) * cdf(d2) / 1e18
    return C
    
    
St = 100e18 # current stock price
K = 100e18
T = 1e18 # time to maturity
r = 0.1e18 # risk free rate
sigma = 0.1e18 # annaul standard deviation , for weiner process

BS_CALL(St, K, T, r, sigma)


In [ ]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)    
    return S * N(d1) - K * np.exp(-r*T)* N(d2)


St = 100 # current stock price
K = 80
T = 1 # time to maturity
r = 0.1 # risk free rate
sigma = 0.1 # annaul standard deviation , for weiner process

#cf_price =  merton_jump_call(St, K, T, r, sigma, np.exp(m+v**2*0.5) , v, lam)

#print('Merton Price =', cf_price)
print('Black Scholes Price =', BS_CALL(St,K,T,r, sigma))

In [ ]:
N(1.05)

In [ ]:
# Heston Parameters
sigma = 0.8 # volatility 
kappa = 0.2 # kappa = ?
theta = 0.1 # theta = ?
volvol = 0.5 # volatility of volatility
rho = 0.4 # rho = ?

St = 100 # current stock price
K = 100
T = 1 # time to maturity
r = 0.1 # risk free rate
m = 0 # meean of jump size
v = 0.3 # standard deviation of jump
lam = 1 # intensity of jump i.e. number of jumps per annum
sigma = 0.1 # annaul standard deviation , for weiner process

#cf_price =  merton_jump_call(St, K, T, r, sigma, np.exp(m+v**2*0.5) , v, lam)

#print('Merton Price =', cf_price)
print('Black Scholes Price =', BS_CALL(St,K,T,r, sigma))

print(d1(St,K,T,r, sigma))

In [ ]:
0.105 - 0.1 * np.sqrt(1)

In [ ]:
np.log(1)

In [ ]:
np.log2(1)

In [20]:
txt = "factorial[{0}] = {1};".format(2,4)
print(txt)

factorial[2] = 4;


In [25]:
for i in range(41):
    
    factorial = np.math.factorial(i)
    
    txt = "factorial[{0}] = {1};".format(i,factorial)
    
    print(txt)
    

factorial[0] = 1;
factorial[1] = 1;
factorial[2] = 2;
factorial[3] = 6;
factorial[4] = 24;
factorial[5] = 120;
factorial[6] = 720;
factorial[7] = 5040;
factorial[8] = 40320;
factorial[9] = 362880;
factorial[10] = 3628800;
factorial[11] = 39916800;
factorial[12] = 479001600;
factorial[13] = 6227020800;
factorial[14] = 87178291200;
factorial[15] = 1307674368000;
factorial[16] = 20922789888000;
factorial[17] = 355687428096000;
factorial[18] = 6402373705728000;
factorial[19] = 121645100408832000;
factorial[20] = 2432902008176640000;
factorial[21] = 51090942171709440000;
factorial[22] = 1124000727777607680000;
factorial[23] = 25852016738884976640000;
factorial[24] = 620448401733239439360000;
factorial[25] = 15511210043330985984000000;
factorial[26] = 403291461126605635584000000;
factorial[27] = 10888869450418352160768000000;
factorial[28] = 304888344611713860501504000000;
factorial[29] = 8841761993739701954543616000000;
factorial[30] = 265252859812191058636308480000000;
factorial[31] = 8222